##Simulation-of-Air-conditioning-processes-in-HVAC-systems


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
DF_data = pd.read_csv('/content/drive/MyDrive/Building /Project 2/data/01.csv', header=0,index_col=0)

In [ ]:
pip install psychrolib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for psychrolib: filename=PsychroLib-2.5.0-py3-none-any.whl size=9776 sha256=755eb3bb38a0c1d55c62a54586480632c0958cc8b2d953a0d3390c377797b557
  Stored in directory: /root/.cache/pip/wheels/0c/7b/62/f3e9bc75ec409442e8854d24b3e8d6b72828c5fd4340d5ecc2
Successfully built psychrolib


In [ ]:
import psychrolib
psychrolib.SetUnitSystem(psychrolib.SI)

In [ ]:
DF_data

,T_db,T_dp,phi,Vdot,BPF
value,,,,,
stream1,30.0,NaN,75,200.0,NaN
coil,NaN,14.0,100,NaN,0.1


#Problem 1 and 2:

Using separate data file

In [ ]:
TDryBulb_stream1 = DF_data.loc['stream1','T_db']
RelHum1 = DF_data.loc['stream1','phi']/100
Pressure = 101325

HumRatio_stream1=psychrolib.GetHumRatioFromRelHum(TDryBulb_stream1, RelHum1, Pressure)
v_stream1 = psychrolib.GetDryAirVolume(TDryBulb_stream1,Pressure)
Enthalpy_stream1 = psychrolib.GetMoistAirEnthalpy(TDryBulb_stream1,HumRatio_stream1)/1000

if 'stream2' in DF_data.index:
  #stream 2
  TDryBulb_stream2 = DF_data.loc['stream2','T_db']
  TDewPoint= DF_data.loc['stream2','T_dp']

  HumRatio_stream2=psychrolib.GetHumRatioFromTDewPoint(TDewPoint, Pressure)
  v_stream2 = psychrolib.GetDryAirVolume(TDryBulb_stream2,Pressure)
  Enthalpy_stream2 = psychrolib.GetMoistAirEnthalpy(TDryBulb_stream2,HumRatio_stream2)/1000

  mass1 = round(DF_data.loc['stream1', 'Vdot']/60/v_stream1,2)
  mass2 = round(DF_data.loc['stream2', 'Vdot']/60/v_stream2,2)

  #stream mix
  Enthalpy_streamMix = round((mass2*Enthalpy_stream2 + mass1*Enthalpy_stream1)/(mass1 + mass2),2)
  HumRatio_streamMix = round((mass2*HumRatio_stream2 + mass1*HumRatio_stream1)/(mass1 + mass2),5)
  """mass_mix = (mass1*HumRatio_stream1 + mass2*HumRatio_stream2)/HumRatio_streamMix
  TDryBulb_streamMix = (mass1*TDryBulb_stream1 + mass2*TDryBulb_stream2)/mass_mix
  TDewPoint_StreamMix = psychrolib.GetTDewPointFromHumRatio(TDryBulb_streamMix,HumRatio_streamMix)"""

  print(f'Problem 1:\n Enthalpy of mixed stream: {Enthalpy_streamMix}\n The Humidity Ratio of mixed stream:{HumRatio_streamMix}')

  #Save
  Q1_Result = {"Mix_Enthalpy":Enthalpy_streamMix , "Mix_Humidity":HumRatio_streamMix }
  DF_Q1 = pd.DataFrame(data=Q1_Result,index=['Mix stream'])
  path_Q1 =os.path.join('/content/drive/MyDrive/Building /Project 2/data', "Q1_result.csv")
  DF_Q1.to_csv(path_Q1)

elif 'coil' in DF_data.index:
  #Coil
  TDewPoint_coil = DF_data.loc['coil', 'T_dp']
  BPF = DF_data.loc['coil','BPF']
  RelHum_coil = DF_data.loc['coil','phi']/100

  T_C = TDewPoint_coil + (TDryBulb_stream1 - TDewPoint_coil)*BPF
  HumRatio_coil = psychrolib.GetHumRatioFromRelHum(T_C,RelHum_coil,Pressure)
  h_c = psychrolib.GetMoistAirEnthalpy(T_C, HumRatio_coil)/1000
  h_w = psychrolib.GetMoistAirEnthalpy(TDryBulb_stream1,HumRatio_coil)/1000

  mass3 = round(DF_data.loc['stream1', 'Vdot']/60/v_stream1,2)

  #Q_calculation
  Q_latent = mass3*(Enthalpy_stream1-h_w)
  Q_sensible = mass3*(h_w-h_c)
  Q_total = Q_latent + Q_sensible
  SHF = Q_sensible/Q_total

  #water
  mass_water = mass3*(HumRatio_stream1-HumRatio_coil)

  print(f'\nProblem 2:\n Q latent: {round(Q_latent,2)}\n Q sensible:{round(Q_sensible,2)}\n Q total:{round(Q_total,2)}\n SHF:{round(SHF,2)}\n Water mass:{round(mass_water,4)}')

  #Save
  Q2_result = {'Q_latent':Q_latent,'Q_sensible':Q_sensible,'Q_total':Q_total, 'SHF':SHF,'mass_water':mass_water}
  DF_Q2 = pd.DataFrame(Q2_result, index=['Q2 Result'])
  path_Q2 = os.path.join('/content/drive/MyDrive/Building /Project 2/data', "Q2_result.csv")
  DF_Q2.to_csv(path_Q2)



Problem 2:
 Q latent: 90.35
 Q sensible:57.36
 Q total:147.71
 SHF:0.39
 Water mass:0.0353


In [ ]:
DF_Q2

,Q_latent,Q_sensible,Q_total,SHF,mass_water
Q2 Result,90.353518,57.358,147.711518,0.388311,0.035339


#Problem 3

In [ ]:
data3 = pd.read_csv('/content/drive/MyDrive/Building /Project 2/data/03_data.csv', header=0,index_col=0)

In [ ]:
DF_data

,temperature,RH,T_db,circulation_condition,circulation_value
stream,,,,,
inside_condition,24.0,0.5,NaN,NaN,NaN
outside_condition,34.0,0.4,NaN,NaN,NaN
air_supply,NaN,NaN,14.0,yes,0.85


In [ ]:
data3

,value
data,
cpm,1.0216
h_fg,2501.0000
Q_latent,100.0000
Q_sensible,400.0000
BPF,0.2000


In [ ]:
Ts = DF_data.loc['air_supply','T_db']
Ti = DF_data.loc['inside_condition','temperature']
To = DF_data.loc['outside_condition', 'temperature']

RH_i = DF_data.loc['inside_condition','RH']
RH_o = DF_data.loc['outside_condition','RH']

cpm = data3.loc['cpm',:]
h_fg = data3.loc['h_fg',:]
Q_latent = data3.loc['Q_latent',:]
Q_sensible = data3.loc['Q_sensible',:]

#calculation
mass_supply = Q_sensible/(Ti-Ts)/cpm
recirculation_condition = DF_data.loc['air_supply', 'circulation_condition']

if recirculation_condition =='yes':
  recirculation_value = DF_data.loc['air_supply', 'circulation_value']
  mass_recirculation = round(mass_supply*recirculation_value,2)
  mass_outside = round(mass_supply-mass_recirculation,2)
  print(f'recirculation value: {recirculation_value*100}%\nmass recirculation:{float(mass_recirculation)} Kg/s\nmass outside:{float(mass_outside)} kg/s')
  Pressure = 101324

  #Outside air condition
  HR_o=psychrolib.GetHumRatioFromRelHum(To, RH_o, Pressure) #Humidity Ratio
  h_o = psychrolib.GetMoistAirEnthalpy(To, HR_o)/1000  #Enthalpy

  print()
  print(f'Outide air Enthalpy: {round(h_o,3)} KJ/Kg\nOutside air Humidity Ratio: {round(HR_o,3)}')

  #recirculation air condition = inside air condition
  HR_i=psychrolib.GetHumRatioFromRelHum(Ti, RH_i, Pressure) #Humidity Ratio
  h_i = psychrolib.GetMoistAirEnthalpy(Ti, HR_i)/1000  #Enthalpy
  T_rec = Ti

  print()
  print(f'recirculation air Enthalpy: {round(h_i,3)} KJ/Kg\nrecirculation air Humidity Ratio: {round(HR_i,3)}')

  HR_s = HR_i-Q_latent/(mass_supply*h_fg)

  # T before coil (T_bc)
  T_bc = (mass_outside*To + mass_recirculation*T_rec)/mass_supply
  HR_bc = (mass_outside*HR_o + mass_recirculation*HR_i)/mass_supply

  Q_sensible_coil = mass_supply*cpm*(T_bc-Ts)
  Q_latent_coil = mass_supply*h_fg*(HR_bc - HR_s)
  Q_total_coil = Q_latent_coil+Q_sensible_coil

  CSHF = Q_sensible_coil/Q_total_coil

  print()
  print(f'Temperature before coil: {round(float(T_bc),2)}\nHR before coil: {round(float(HR_bc),4)}\n\
  Q_sensible_coil:{round(float(Q_sensible_coil),2)}\nQ_latent_coil: {round(float(Q_latent_coil),2)}\n\
  Q_total_coil: {round(float(Q_total_coil),2)}\nCSHF: {round(float(CSHF),2)}')

  #Save
  Q3_rec_result = { 'mass_recirculation':float(mass_recirculation),
                    'mass_outside':float(mass_outside),
                    "Outide air Enthalpy":round(float(h_o),5),
                    'Outside air Humidity Ratio':round(float(HR_o),5),
                    'recirculation air Enthalpy':round(float(h_i),5),
                    'recirculation Humidity Ratio':round(float(HR_i),5),
                    'Temperature before coil':round(float(T_bc),2),
                    'HR before coil': round(float(HR_bc),2),
                    'Q_sensible_coil':round(float(Q_sensible_coil),2),
                    'Q_latent_coil':round(float(Q_latent_coil),2),
                    'Q_total_coil':round(float(Q_total_coil),2),'CSHF':round(float(CSHF),2)}
  DF_Q3_rec = pd.DataFrame(Q3_rec_result, index=['Q3_85%_Cir'])
  path_Q3_rec = os.path.join('/content/drive/MyDrive/Building /Project 2/data', "Q3_rec_result.csv")
  DF_Q3_rec.to_csv(path_Q3_rec)

else:
  #Outside air condition
  HR_o=psychrolib.GetHumRatioFromRelHum(To, RH_o, Pressure) #Humidity Ratio
  h_o = psychrolib.GetMoistAirEnthalpy(To, HR_o)/1000  #Enthalpy

  print()
  print(f'Outide air Enthalpy: {round(h_o,3)} KJ/Kg\nOutside air Humidity Ratio: {round(HR_o,3)}')

  # inside air condition
  HR_i=psychrolib.GetHumRatioFromRelHum(Ti, RH_i, Pressure) #Humidity Ratio
  h_i = psychrolib.GetMoistAirEnthalpy(Ti, HR_i)/1000  #Enthalpy

  #supply condition
  HR_s = HR_i-Q_latent/(mass_supply*h_fg)

  print()
  print(f'inside air Enthalpy: {round(h_i,3)} KJ/Kg\ninsideHumidity Ratio: {round(HR_i,3)}\nSupply HR:{round(float(HR_s),5)}')

  #Load Calculation
  Q_sensible_coil = mass_supply*cpm*(To-Ts)
  Q_latent_coil = mass_supply*h_fg*(HR_o - HR_s)
  Q_total_coil = Q_latent_coil+Q_sensible_coil
  CSHF = Q_sensible_coil/Q_total_coil

  print()
  print(f'Q_sensible_coil:{round(float(Q_sensible_coil),2)}\nQ_latent_coil: {round(float(Q_latent_coil),2)}\nQ_total_coil: {round(float(Q_total_coil),2)}\n\nCSHF: {round(float(CSHF),2)}')

  BPF = data3.loc['BPF',:]
  T_ADP = (BPF*To-Ts)/(BPF-1)
  print(f'BPF:{round(float(BPF),2)}')

  #save
  Q3_fresh_result={"Outide air Enthalpy": h_o,
                   'Outside air Humidity Ratio':HR_o,
                   'inside air Enthalpy':h_i,
                   'insideHumidity Ratio':HR_i,
                   'Supply HR':HR_s,'Q_sensible_coil':float(Q_sensible_coil),
                   'Q_latent_coil':float(Q_latent_coil),
                   'Q_total_coil':float(Q_total_coil),'CSHF':float(CSHF), 'BPF':float(BPF) }
  DF_Q3_fresh = pd.DataFrame(Q3_fresh_result, index=['Q3_fresh_air'])
  path_Q3_fresh = os.path.join('/content/drive/MyDrive/Building /Project 2/data', "Q3_fresh_result.csv")
  DF_Q3_fresh.to_csv(path_Q3_fresh)


recirculation value: 85.0%
mass recirculation:33.28 Kg/s
mass outside:5.87 kg/s

Outide air Enthalpy: 68.442 KJ/Kg
Outside air Humidity Ratio: 0.013

recirculation air Enthalpy: 47.815 KJ/Kg
recirculation air Humidity Ratio: 0.009

Temperature before coil: 25.5
HR before coil: 0.0099
  Q_sensible_coil:459.86
Q_latent_coil: 159.41
  Q_total_coil: 619.28
CSHF: 0.74


In [ ]:
Q3_rec_result

{'mass_recirculation': 33.28,
 'mass_outside': 5.87,
 'Outide air Enthalpy': 68.44247563592901,
 'Outside air Humidity Ratio': 0.01335,
 'recirculation air Enthalpy': 47.81488,
 'recirculation Humidity Ratio': 0.0093,
 'Temperature before coil': 25.5,
 'HR before coil': 0.01,
 'Q_sensible_coil': 459.86,
 'Q_latent_coil': 159.41,
 'Q_total_coil': 619.28,
 'CSHF': 0.74}